In [1]:
import nltk
# nltk.download('all')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import warnings
import tensorflow
warnings.filterwarnings('ignore')

#preprocessing

In [27]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [28]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent["tag"]))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [30]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
print (len(documents), "documents")
classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

628 documents
1 classes {'intents': [{'tag': 'greetings1', 'patterns': ['hi', 'how are you?', 'is anyone there?', 'hello', 'good day', "what's up", 'how are you?', 'how are ya?', 'heyy', 'whatsup', '??? ??? ??', 'hello ', 'hello bot', 'hey'], 'responses': ['Hello! How can I help you today?', 'Hello :)'], 'context_set': '', 'links': []}, {'tag': 'greetings2', 'patterns': ['How are you?', 'How are you doing?', "How's it going?", 'How are you feeling today?', "What's up?", "How's everything?", "How's your day?", 'How are things with you?', "What's new?", "How's life treating you?", "How's your day going so far?", "What's happening?", 'How are you holding up?', "What's going on in your world?"], 'responses': ["I'm fine what about you?", "Thanks for asking! I'm excellent. How about you?", 'Fine, thanks. What do you got for me today?'], 'context_set': '', 'links': []}, {'tag': 'name', 'patterns': ['name', 'your name', 'do you have a name', 'what are you called', 'what is your name', 'what sh

In [31]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [32]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

AttributeError: 'dict' object has no attribute 'index'

In [14]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("chat_model.h5")

Epoch 1/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0621 - loss: 2.8871 
Epoch 2/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1345 - loss: 2.7301
Epoch 3/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2280 - loss: 2.5304
Epoch 4/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3405 - loss: 2.2802
Epoch 5/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3850 - loss: 2.0541
Epoch 6/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4799 - loss: 1.8097
Epoch 7/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4731 - loss: 1.7186
Epoch 8/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5355 - loss: 1.5162
Epoch 9/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6558 - loss: 1.2289
Epoch 10/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6738 - loss: 1.0696
Epoch 11/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6342 - loss: 1.1414
Epoch 12/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy